<a href="https://colab.research.google.com/github/vivianagiorgi/ANN_homework/blob/main/Copia_BASE_HW_(Data_Augmentation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/HOMEWORK_ANN_1

/gdrive/MyDrive/HOMEWORK_ANN_1


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.7.0


In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# !unzip dataset.zip

In [ ]:
dataset_dir = 'training'

In [ ]:
# Plot example images from dataset
labels = ['Apple','Blueberry','Cherry','Corn','Grape','Orange','Peach','Pepper','Potato','Raspberry','Soybean','Squash','Strawberry','Tomato']

#num_row = len(labels)//2
#num_col = len(labels)//num_row
#fig, axes = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))
#for i in range(len(labels)):
  #if i < len(labels):
    #class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
    #class_img = class_imgs[0]
    #img = Image.open('{}/{}/{}'.format(dataset_dir, labels[i], class_img))
    #ax = axes[i//num_col, i%num_col]
    #ax.imshow(np.array(img))
    #ax.set_title('{}'.format(labels[i]))
#plt.tight_layout()
#plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_gen = ImageDataGenerator()
# valid_data_gen = ImageDataGenerator()

**WITHOUT DATA AUGMENTATION**

In [ ]:
noaug_train_data_gen = ImageDataGenerator(rescale=1/255.)
# validation and test CANNOT BE AUGMENTED
noaug_train_gen = noaug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                                           target_size=(256,256),
                                                           color_mode='rgb',
                                                           classes=None, # can be set to labels
                                                           class_mode='categorical',
                                                           batch_size=100,
                                                           shuffle=True,
                                                           seed=seed)

Found 17728 images belonging to 14 classes.


In [ ]:
print("Assigned labels")
print(noaug_train_gen.class_indices)
print()
print("Target classes")
print(noaug_train_gen.classes)

Assigned labels
{'Apple': 0, 'Blueberry': 1, 'Cherry': 2, 'Corn': 3, 'Grape': 4, 'Orange': 5, 'Peach': 6, 'Pepper': 7, 'Potato': 8, 'Raspberry': 9, 'Soybean': 10, 'Squash': 11, 'Strawberry': 12, 'Tomato': 13}

Target classes
[ 0  0  0 ... 13 13 13]


**WITH DATA AUGMENTATION**

NB: c'è tutta una parte che non ho aggiunto, credo serva solo per vedere cosa fa il data augmentation in una immagine nello specifico

In [ ]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=30,
                                        height_shift_range=50,
                                        width_shift_range=50,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True, 
                                        fill_mode='reflect',
                                        rescale=1/255.) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                                       target_size=(256,256),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=100,
                                                       shuffle=True,
                                                       seed=seed)

Found 17728 images belonging to 14 classes.


In [ ]:
input_shape = (256, 256, 3)
epochs = 10

In [ ]:
# Model used for the exercise:
# (Conv + ReLU + MaxPool) x 5 + FC x 2
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv5)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool5)
    flattening_layer = tfkl.Dropout(0.3, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=512, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.3, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # ti salva il modello fino a questo punto (creato da lui in modo che funzioni così)
  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, # PLOTTA SU TENSORBOARD
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [ ]:
model = build_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 64)        18496 

TRAINING WITH **NO** DATA AUGM

In [ ]:
# Create folders and callbacks and fit
noaug_callbacks = create_folders_and_callbacks(model_name='CNN_NoAug')

# Train the model
history = model.fit(
    x = noaug_train_gen,
    epochs = epochs,
    callbacks = noaug_callbacks,
).history

Epoch 1/50
178/178 [==============================] - ETA: 0s - loss: 1.0917 - accuracy: 0.6662 

INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Nov15_12-26-51/ckpts/cp.ckpt/assets


INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Nov15_12-26-51/ckpts/cp.ckpt/assets


178/178 [==============================] - 2041s 12s/step - loss: 1.0917 - accuracy: 0.6662
Epoch 2/50
178/178 [==============================] - ETA: 0s - loss: 0.4431 - accuracy: 0.8590

INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Nov15_12-26-51/ckpts/cp.ckpt/assets


...

INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Nov15_12-26-51/ckpts/cp.ckpt/assets


178/178 [==============================] - 78s 438ms/step - loss: 0.0137 - accuracy: 0.9956
Epoch 40/50
178/178 [==============================] - ETA: 0s - loss: 0.0217 - accuracy: 0.9935

INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Nov15_12-26-51/ckpts/cp.ckpt/assets


INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Nov15_12-26-51/ckpts/cp.ckpt/assets


178/178 [==============================] - 70s 394ms/step - loss: 0.0217 - accuracy: 0.9935


In [ ]:
# Save best epoch model
model.save("data_augmentation_experiments/CNN_NoAug_Best2")

INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Best/assets


INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_NoAug_Best/assets


In [ ]:
# Build model (for data augmentation training)
model = build_model(input_shape)
# che senso ha rifarlo??

In [ ]:
# da mettere prima o dopo il training?

tf.get_logger().setLevel('WARNING') #  if you want to suppress only INFOs
# tf.get_logger().setLevel('ERROR') #  if you want to suppress both WARNINGs and INFOs

# Create folders and callbacks and fit
aug_callbacks = create_folders_and_callbacks(model_name='CNN_Aug')

# Train the model
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    callbacks = aug_callbacks,
).history

Epoch 1/10
127/178 [====================>.........] - ETA: 15:40 - loss: 1.6818 - accuracy: 0.4707

In [ ]:
# Save best epoch model
model.save("data_augmentation_experiments/CNN_Aug_Best")

### + tutta la parte di VISUALIZE ACTIVATIONS non so se vogliamo metterla

## + tecniche per prevenire overfitting / tecniche di regularization hanno senso in questi casi di classificazione immagini? sono in ANN #2